<a href="https://colab.research.google.com/github/yashkapur0403/Lung-X-Ray-Pnuemonia/blob/main/log_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import cv2
import numpy as np
import zipfile

# Unzip the dataset from Google Drive
zip_path = "/content/drive/My Drive/Colab Notebooks/chest_xray.zip"
extract_path = "/content/chest_xray"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Function to load images and labels
def load_images_from_folder(folder_path, label):
    data = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (64, 64))          # Resize to 64x64
            img_flat = img.flatten() / 255.0         # Normalize and flatten
            data.append((img_flat, label))
    return data

In [5]:

normal_data = load_images_from_folder("/content/chest_xray/chest_xray/train/NORMAL", 0)
pneumonia_data = load_images_from_folder("/content/chest_xray/chest_xray/train/PNEUMONIA", 1)
all_data = normal_data + pneumonia_data

X_train = np.array([x for x, _ in all_data])
y_train = np.array([y for _, y in all_data]).reshape(-1, 1)

normal_test = load_images_from_folder("/content/chest_xray/chest_xray/test/NORMAL", 0)
pneumonia_test = load_images_from_folder("/content/chest_xray/chest_xray/test/PNEUMONIA", 1)
all_test = normal_test + pneumonia_test

X_test = np.array([x for x, _ in all_test])
y_test = np.array([y for _, y in all_test])

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def loss(y, a):
    # Binary cross-entropy
    return -np.mean(y * np.log(a) + (1 - y) * np.log(1 - a))

def train(X, y, lr=0.01, epochs=100):
    m, n = X.shape
    w = np.zeros((n, 1))
    b = 0

    for epoch in range(epochs):
        z = np.dot(X, w) + b
        a = sigmoid(z)

        dw = np.dot(X.T, (a - y)) / m
        db = np.sum(a - y) / m

        w -= lr * dw
        b -= lr * db

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss(y, a)}")
    return w, b


In [7]:
weights, bias = train(X_train, y_train, lr=0.01, epochs=100)

def predict(X, w, b):
    z = np.dot(X, w) + b
    return (sigmoid(z) > 0.5).astype(int)

# Training accuracy
train_preds = predict(X_train, weights, bias)
train_acc = np.mean(train_preds == y_train) * 100
print(f"Training Accuracy: {train_acc:.2f}%")

# Test accuracy
test_preds = predict(X_test, weights, bias)
test_acc = np.mean(test_preds == y_test) * 100
print(f"Test Accuracy: {test_acc:.2f}%")

Epoch 0, Loss: 0.6931471805599453
Epoch 10, Loss: 0.5459928130165052
Epoch 20, Loss: 0.5000304631982598
Epoch 30, Loss: 0.4594497881763436
Epoch 40, Loss: 0.4247242577529554
Epoch 50, Loss: 0.39741883120986127
Epoch 60, Loss: 0.3771428684542337
Epoch 70, Loss: 0.3609670176731732
Epoch 80, Loss: 0.34715508296476427
Epoch 90, Loss: 0.3351257013027529
Training Accuracy: 88.47%
Test Accuracy: 60.18%


In [2]:
# Re-run this cell (alone) to re-mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive
